In [ ]:
import pandas as pd
import re
from unidecode import unidecode
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import datetime
import time
USERNAME = "hello"
PASSWORD = "here"
EXPORT_DIR = r"C:\Users\telesales\Documents\export mẫu"
DOWNLOAD_MODE = "BOTH"
FILTER_FROM = None
FILTER_TO   = None
BASE_URL    = "https://spo.hvbl.vn/en/admin"
LOGIN_PAGE  = f"{BASE_URL}/login"
LOGIN_API   = f"{BASE_URL}/ajax/login"
ORDERS_PAGE = f"{BASE_URL}/orders"
API_FILTER  = f"{BASE_URL}/ajax/request/export-filter"
API_TYPE    = f"{BASE_URL}/ajax/request/export"
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0",
    "X-Requested-With": "XMLHttpRequest"
})
def clear_old_csv_files(folder):
    deleted = 0
    for f in os.listdir(folder):
        if f.lower().endswith(".csv"):
            try:
                os.remove(os.path.join(folder, f))
                deleted += 1
            except Exception as e:
                print(f"⚠️ Cannot delete {f}: {e}")
def get_timestamp_filename():
    return f"FORM_REQUEST_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.csv"
def get_latest_csv(folder):
    files = [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if f.lower().endswith(".csv")
    ]
    if not files:
        raise RuntimeError("❌ No CSV file found")
    return max(files, key=os.path.getctime)
def download_file(api_url, params):
    filename = get_timestamp_filename()
    full_path = os.path.join(EXPORT_DIR, filename)

    r = session.post(api_url, params=params, stream=True)
    if r.status_code != 200:
        raise RuntimeError(f"❌ Export failed ({r.status_code})")
    with open(full_path, "wb") as f:
        for chunk in r.iter_content(8192):
            f.write(chunk)
    return full_path
def main():
    os.makedirs(EXPORT_DIR, exist_ok=True)
    clear_old_csv_files(EXPORT_DIR)
    r_login = session.get(LOGIN_PAGE)
    soup = BeautifulSoup(r_login.text, "html.parser")
    token = soup.find("input", {"name": "_token"})
    if not token:
        raise RuntimeError("❌ Login token not found")
    login_resp = session.post(LOGIN_API, data={
        "_token": token["value"],
        "username": USERNAME,
        "password": PASSWORD
    })
    if login_resp.status_code != 200:
        raise RuntimeError("❌ Login failed")
    r_orders = session.get(ORDERS_PAGE)
    soup_orders = BeautifulSoup(r_orders.text, "html.parser")
    csrf = soup_orders.find("meta", {"name": "csrf-token"})
    if csrf:
        session.headers.update({
            "x-csrf-token": csrf["content"],
            "Referer": ORDERS_PAGE
        })
    rawsup = None
    rawspo = None
    if DOWNLOAD_MODE in ("BOTH", "RAW_FILTER"):
        params = {"length_request": "100000"}
        if FILTER_FROM and FILTER_TO:
            params["filter_from_date"] = FILTER_FROM
            params["filter_to_date"]   = FILTER_TO
        download_file(API_FILTER, params)
        time.sleep(1)
        latest = get_latest_csv(EXPORT_DIR)
        rawsup = pd.read_csv(latest, encoding="utf-8-sig")
        rawsup["SPO#"] = rawsup["SPO#"].astype(str).str.strip()
        rawsup = rawsup.drop_duplicates(subset=["SPO#"], keep="first").reset_index(drop=True)
    if DOWNLOAD_MODE in ("BOTH", "RAW_FULL"):
        download_file(API_TYPE, {"type": "1"})
        time.sleep(1)
        latest = get_latest_csv(EXPORT_DIR)
        rawspo = pd.read_csv(latest, encoding="utf-8-sig")
    return rawsup, rawspo
if __name__ == "__main__":
    rawsup, rawspo = main()
item_map = pd.read_excel( r"C:\Users\telesales\Downloads\ratecard.xlsx")
rawsup = rawsup.drop_duplicates(
    subset=["SPO#"],
    keep="first"
).reset_index(drop=True)
os.remove("raw xu li chi phi.csv")
os.remove('raw xu li.csv')
rawspo['Cost'] = pd.to_numeric(
    rawspo['Cost'].astype(str).str.replace(',', '', regex=False),
    errors='coerce'
)
cols_to_drop = [
    'Variation', 'Completed',
    'Actual Installed Signage',
    'OSR#', 'Type of license', 'SR Accepted Date', 'Budget'
]
rawspo = rawspo.drop(columns=cols_to_drop, errors='ignore')
rawspo.reset_index(drop=True, inplace=True)
cols_to_fill = [
    'SPO#', 'Request Date', 'Type', 'Requester', 'OutletID',
    'Outlet Name', 'Status', 'Reason', 'License#',
    'Signage Class', 'Effective Date Start', 'Effective Date End',
    'Supplier', 'SS/SE', 'Sales Rep','Pending at'
]
first_col = cols_to_fill[0]
header_mask = rawspo[first_col].notna()
next_first_blank = rawspo[first_col].isna().shift(-1, fill_value=False)
to_drop = header_mask & next_first_blank

rawspo[cols_to_fill] = rawspo[cols_to_fill].ffill()
rawspo = rawspo[~to_drop].reset_index(drop=True)
rawspo['Signage Type'] = rawspo['Signage Type'].replace('', pd.NA).fillna('variation')
rawspo['Request Date'] = pd.to_datetime(rawspo['Request Date'], dayfirst=True, errors='coerce')
start_date = pd.Timestamp('2026-01-01')
latest_date = rawspo['Request Date'].max()
rawspo = rawspo[
    (rawspo['Request Date'] >= start_date) &
    (rawspo['Request Date'] <= latest_date)
]
for col in rawspo.select_dtypes(include='object').columns:
    rawspo[col] = rawspo[col].apply(lambda x: unidecode(str(x)) if pd.notna(x) else x)
rawspo.loc[rawspo['Cost'].isin(['-', '']), 'Cost'] = pd.NA
rawspo['Cost'] = (
    pd.to_numeric(rawspo['Cost'], errors='coerce')
      .fillna(0)
      .astype(int)
)
for col in ['Item', 'Signage Type', 'Brand']:
    rawspo[col] = rawspo[col].astype(str).str.lower().str.strip()
signage_to_cost_type = {
    'hiflex sign': 'cp_bang',
    'chi phi chuyen bang hieu va lap dat': 'cp_nhancong',
    'lightbox (standard - hop den thuong)': 'cp_bang',
    'new license (giay phep moi)': 'cp_giayphep',
    'renewal license (giay phep gia han)': 'cp_giayphep',
    'metal accessories (vat lieu kim loai)': 'cp_vattu',
    'hiflex sign (bang hieu bat hiflex)': 'cp_bang',
    'electric items (linh kien dien)': 'cp_vattu',
    'flex items (cac loai bat thay the)': 'cp_vattu',
    'lightbox (premium - hop den led logo)': 'cp_bang',
    'other accessories (decal, kinh cuong luc,...)': 'cp_vattu',
    '4.0 - sat': 'cp_vattu',
    'sat': 'cp_vattu',
    'variation': 'cp_khac',
    'canopy (mai hien)': 'cp_vattu',
    'transportation costs (chi phi van chuyen)': 'cp_vanchuyen',
    'labor cost (chi phi nhan cong)': 'cp_nhancong',
    'hiflex shopname (bang hieu bat hiflex ten outlet)': 'cp_vattu',
    'sat tru': 'cp_vattu',
    'tole sign (ton)': 'cp_vattu',
    'other sign (alu, mica, tole...)': 'cp_vattu',
    'lightbox (standard - shop name)': 'cp_vattu',
    'other cost (cac chi phi phat sinh khac)': 'cp_khac',
    'lightbox shopname (hop den ten outlet)': 'cp_vattu',
    '4.0 - new permit': 'cp_giayphep',
    'cp van chuyen': 'cp_vanchuyen',
    'day dien': 'cp_vattu',
    '4.0 - light': 'cp_vattu',
    '4.0 - cp nhan cong': 'cp_nhancong',
    'led logo': 'cp_bang',
    'light': 'cp_vattu',
    'storage cost (chi phi luu kho)': 'cp_khac',
    '4.0 - cp van chuyen': 'cp_vanchuyen',
    'mica sign': 'cp_vattu',
    'cp nhan cong': 'cp_nhancong',
    '4.0 - day dien': 'cp_vattu',
    '4.0 - tole (ton)': 'cp_vattu',
    'hiflex sign (shop name)': 'cp_vattu',
    'bo nguon': 'cp_vattu',
    'giay phep moi': 'cp_giayphep',
    'repair cost (chi phi sua chua)': 'cp_nhancong',
    'giay phep gia han': 'cp_giayphep',
    '4.0 - hiflex sign': 'cp_bang',
    'special design (emblem, standee gate, booth, wall, menu...)': 'cp_vattu'
}
rawspo['cost_type'] = rawspo['Signage Type'].map(lambda x: signage_to_cost_type.get(str(x).strip().lower(), 'cp_khac'))
# normalize text để map không lỗi
for col in ['Item', 'loai item']:
    item_map[col] = (
        item_map[col]
        .astype(str)
        .str.lower()
        .str.strip()
    )

rawspo = rawspo.merge(
    item_map[['Item', 'loai item']],
    on='Item',
    how='left'
)
led_item_force_1 = item_map.loc[
    item_map['led quantities'] == 1,
    'Item'
].unique()

rawspo.loc[
    rawspo['Item'].isin(led_item_force_1),
    'Quantity'
] = 1

# ===== 5. STEP 2: LED logo đã map nhưng Quantity không phải số nguyên → Quantity = 1 =====
led_mapped_mask = rawspo['Item'].isin(item_map['Item'])

invalid_qty_mask = rawspo['Quantity'].isna() | (
    rawspo['Quantity'] % 1 != 0
)

rawspo.loc[
    led_mapped_mask & invalid_qty_mask,
    'Quantity'
] = 1

rawspo['Quantity'] = rawspo['Quantity'].fillna(1).astype(int)
rawspo.to_csv("raw xu li chi phi.csv", index=False)

C:\Users\telesales\AppData\Local\Temp\ipykernel_5096\1418309407.py:91: DtypeWarning: Columns (15,25) have mixed types. Specify dtype option on import or set low_memory=False.
  rawsup = pd.read_csv(latest, encoding="utf-8-sig")
C:\Users\telesales\AppData\Local\Temp\ipykernel_5096\1418309407.py:98: DtypeWarning: Columns (8,22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  rawspo = pd.read_csv(latest, encoding="utf-8-sig")


In [2]:
cost_cols = ['CP_Bang','CP_GiayPhep','CP_VanChuyen','CP_VatTu','CP_Khac','CP_NhanCong']
for c in cost_cols:
    rawspo[c] = 0

rawspo.loc[rawspo['cost_type']=='cp_bang',     'CP_Bang']      = rawspo['Cost']
rawspo.loc[rawspo['cost_type']=='cp_giayphep', 'CP_GiayPhep']  = rawspo['Cost']
rawspo.loc[rawspo['cost_type']=='cp_vanchuyen','CP_VanChuyen'] = rawspo['Cost']
rawspo.loc[rawspo['cost_type']=='cp_vattu',    'CP_VatTu']     = rawspo['Cost']
rawspo.loc[rawspo['cost_type']=='cp_khac',     'CP_Khac']      = rawspo['Cost']
rawspo.loc[rawspo['cost_type']=='cp_nhancong', 'CP_NhanCong']  = rawspo['Cost']
rawspo['Total_Cost']     = rawspo[cost_cols].sum(axis=1)
rawspo['ten_loai_bang_hieu'] = rawspo.apply(
    lambda r: r['Item'] if r['cost_type']=='cp_bang' else '', axis=1
)
brand_keywords = sorted([
    'tiger', 'heineken', 'bia viet', 'tiger crystal',
    'heineken silver', 'larue special', 'heineken 0.0', 'larue smooth', 'larue',
    'biere la rue', 'bivina', 'bivina export', 'larue lemon',
    'edelweiss', 'tiger soju', 'strongbow'
])
def extract_brand(text):
    text = str(text).lower()
    for kw in brand_keywords:
        if kw in text:
            return kw.title()
    return ''
lead_brand_map = rawsup.set_index('SPO#')['Lead Brand']
rawspo['Lead Brand'] = rawspo['SPO#'].map(lead_brand_map)
rawspo['Brand_extracted'] = rawspo['Brand'].apply(extract_brand)
rawspo['LeadBrand_extracted'] = rawspo['Lead Brand'].apply(extract_brand)
def determine_spo_brand(group):
    non_null_brand = group['Brand_extracted'][group['Brand_extracted'] != '']
    if not non_null_brand.empty:
        return non_null_brand.iloc[0]
    non_null_lead = group['LeadBrand_extracted'][group['LeadBrand_extracted'] != '']
    if not non_null_lead.empty:
        return non_null_lead.iloc[0]
    return ''
spo_brand_map = rawspo.groupby('SPO#').apply(determine_spo_brand).reset_index(name='SPO_brand')
rawspo = rawspo.merge(spo_brand_map, on='SPO#', how='left')
meta_cols = [
    'Request Date', 'Type', 'Requester', 'OutletID', 'Outlet Name',
    'Status', 'Signage Class', 'Supplier', 'SS/SE', 'Sales Rep',
    'SPO_brand',
]
agg_dict = {
    **{c: 'sum' for c in cost_cols},
    'Total_Cost': 'sum',
    'ten_loai_bang_hieu': lambda x: ', '.join(sorted(set(filter(None, x)))),
    'SPO_brand': 'first',
    'Request Date': 'first',
    'OutletID': 'first'
}
agg_dict.update({col: 'first' for col in meta_cols})
grouped_rawspo = rawspo.groupby('SPO#', as_index=False).agg(agg_dict)
item_types = ["bang hiflex", "bat tha hiflex", "canopy", "hop den hiflex",
              "led logo", "bat hiflex", "tranh dien", "emblem"]
normal_items = [t for t in item_types if t != "led logo"]
normal_counts = (
    rawspo[rawspo['loai item'].isin(normal_items)]
    .groupby(['SPO#', 'loai item'])
    .size()
    .unstack(fill_value=0)
)
led_logo_counts = (
    rawspo[rawspo['loai item'] == 'led logo']
    .groupby(['SPO#'])['Quantity']
    .sum()
    .to_frame('led logo')
)
item_counts = normal_counts.join(led_logo_counts, how='outer').fillna(0)
for t in item_types:
    if t not in item_counts.columns:
        item_counts[t] = 0
item_counts = (
    item_counts[item_types]
    .rename(columns={t: f"item_{t.replace(' ', '_')}" for t in item_types})
    .reset_index()
)
grouped_rawspo.rename(columns={'Status': 'status_SPO'}, inplace=True)
grouped_rawspo = grouped_rawspo.merge(item_counts, on="SPO#", how="left").fillna(0)
bang_item_cols = [
    'item_bang_hiflex',
    'item_bat_tha_hiflex',
    'item_canopy',
    'item_hop_den_hiflex',
    'item_led_logo',
    'item_bat_hiflex',
    'item_tranh_dien',
    'item_emblem'
]
grouped_rawspo[bang_item_cols] = grouped_rawspo[bang_item_cols].astype(int)
grouped_rawspo['item_led'] = grouped_rawspo['item_led_logo']
grouped_rawspo['item_bang'] = grouped_rawspo[bang_item_cols].sum(axis=1)
grouped_rawspo['item_standard'] = (
    grouped_rawspo['item_bang'] - grouped_rawspo['item_led']
)
count_cols = [c for c in grouped_rawspo.columns if c.startswith("item_")]
grouped_rawspo[count_cols] = grouped_rawspo[count_cols].astype(int)
grouped_rawspo['SPO#'] = grouped_rawspo['SPO#'].astype(str).str.strip()
rawsup['SPO#'] = rawsup['SPO#'].astype(str).str.strip()
rawsup_idx = rawsup.set_index('SPO#')
cols_to_map = [
    'Supplier',
    'Status',
    'Area',
    'Region',
    'ASM Approve ',
    'SR Accepted Date',
    'Budget'
]
for col in cols_to_map:
    if col in rawsup_idx.columns:
        grouped_rawspo[col] = grouped_rawspo['SPO#'].map(rawsup_idx[col])
    else:
        print(f"⚠️ Missing column in rawsup: {col}")
grouped_rawspo['Complete/In progress'] = 'In progress'
grouped_rawspo.loc[grouped_rawspo['Status'].str.contains('cancel', case=False, na=False),
                   'Complete/In progress'] = 'Cancelled'
grouped_rawspo.loc[grouped_rawspo['Status'].str.contains('install|accept|finish', case=False, na=False),
                   'Complete/In progress'] = 'Complete'
grouped_rawspo['ASM Status'] = 'In progress'
grouped_rawspo.loc[grouped_rawspo['Status'].str.contains('cancel', case=False, na=False),
                   'ASM Status'] = 'Cancelled'
grouped_rawspo.loc[grouped_rawspo['Status'].str.contains('install|accept|finish', case=False, na=False),
                   'ASM Status'] = 'Complete'
grouped_rawspo.loc[grouped_rawspo['Status'].str.contains('approved|variation|revised', case=False, na=False),
                   'ASM Status'] = 'ASM approve'
grouped_rawspo['Pending At'] = 'In progress'
grouped_rawspo.loc[grouped_rawspo['Status'].str.contains('cancel', case=False, na=False),
                   'Pending At'] = 'Cancelled'
grouped_rawspo.loc[grouped_rawspo['Status'].str.contains('install|accept|finish', case=False, na=False),
                   'Pending At'] = 'Complete'
grouped_rawspo.loc[
    grouped_rawspo['Status'].str.contains('submitted|checked marquette|approved|full checked|rejected',
                                          case=False, na=False)
    & (grouped_rawspo['Pending At'] == 'In progress'),
    'Pending At'
] = 'Sale team'
grouped_rawspo.loc[
    grouped_rawspo['Status'].str.contains('verified|variation|revised', case=False, na=False)
    & (grouped_rawspo['Pending At'] == 'In progress'),
    'Pending At'
] = 'nha ve'
grouped_rawspo['Warchest/AP'] = 'AP'
warchest_mask = ( grouped_rawspo['Budget'].str.contains('warchest', case=False, na=False) & ~grouped_rawspo['Budget'].str.contains('ca mau', case=False, na=False))
grouped_rawspo.loc[warchest_mask & grouped_rawspo['Budget'].str.contains('tiger', case=False, na=False), 'Warchest/AP'] = 'Warchest TIGER'
grouped_rawspo.loc[warchest_mask & grouped_rawspo['Budget'].str.contains('heineken', case=False, na=False), 'Warchest/AP'] = 'Warchest HEINEKEN'
grouped_rawspo.loc[warchest_mask & grouped_rawspo['Budget'].str.contains('bivina', case=False, na=False), 'Warchest/AP'] = 'Warchest BIVINA'
grouped_rawspo.loc[grouped_rawspo['Budget'].str.contains('wargame', case=False, na=False), 'Warchest/AP'] = 'Warchest WARGAME'
grouped_rawspo.rename(columns={
    'Status': 'status_SPO',
    'item_Bang_Hiflex': 'item_bang_hiflex',
    'item_Bat_tha_Hiflex': 'item_bat_tha_hiflex',
    'item_Canopy': 'item_canopy',
    'item_Hop_Den_Hiflex': 'item_hop_den_hiflex',
    'item_Led_Logo': 'item_led_logo',
    'item_Bat_Hiflex': 'item_bat_hiflex',
    'item_Tranh_dien': 'item_tranh_dien',
    'item_Emblem': 'item_emblem',
    'ASM Approve ': 'ASM Approve'
}, inplace=True)
final_cols = [
    'SPO#','CP_Bang','CP_GiayPhep','CP_VanChuyen','CP_VatTu','CP_Khac','CP_NhanCong',
    'Total_Cost','ten_loai_bang_hieu','SPO_brand','Request Date','OutletID',
    'Type','Requester','Outlet Name','status_SPO','Signage Class','Supplier',
    'SS/SE','Sales Rep',
    'item_bang_hiflex','item_bat_tha_hiflex','item_canopy','item_hop_den_hiflex',
    'item_led_logo','item_bat_hiflex','item_tranh_dien','item_emblem',
    'item_led','item_bang','item_standard',
    'Status','Area','Region','ASM Approve','SR Accepted Date','Budget',
    'Complete/In progress','ASM Status','Pending At','Warchest/AP'
]
for c in final_cols:
    if c not in grouped_rawspo.columns:
        grouped_rawspo[c] = 0
grouped_rawspo = grouped_rawspo[final_cols]

grouped_rawspo.to_csv('raw xu li.csv', index=False, encoding='utf-8-sig')


C:\Users\telesales\AppData\Local\Temp\ipykernel_5096\1148372807.py:39: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spo_brand_map = rawspo.groupby('SPO#').apply(determine_spo_brand).reset_index(name='SPO_brand')
